# Knowledge and Data: Practical Assignment 3 
## RDF Data, RDFS knowledge and inferencing 

YOUR NAME: Thomas Norton

YOUR VUNetID: tnn281

*(If you do not provide your name and VUNetID we will not accept your submission).* 

### Learning objectives

At the end of this exercise you should be able to:

1. Access local an external data via SPARQL both from within a python programming environment and stand-alone with a GUI, such as [YASGUI](https://yasgui.triply.cc/), and this way integrate data from different sources  
2. Model your own first knowledge base, in this case an RDF Schema knowledge graph
3. Implement inference rules 

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, safe the Notebook and submit it 
as a Jupyter Notebook, i.e. with an ipynb extension. Please use as name of the 
Notebook your studentID+Assignment3.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

Before you start, you need to:

- **Install the *rdflib* Python package:** *pip install rdflib* (should already be installed from the previous assignment)
- **Install the *SPARQLWrapper* Python package:** *pip install SPARQLWrapper*
- **Install the free edition of the GraphDB Triplestore:** please follow this short [GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md). 

Then, add the file example-from-slides.ttl to a newly created database, say called assignment-3. 

**Note that you should have an active internet connection to run the code in this notebook.**

In [4]:
# install library
%pip install SPARQLWrapper

You should consider upgrading via the 'c:\Python310\python.exe -m pip install --upgrade pip' command.


## Task 1: (3 points) Integrate Local and External Data

You can integrate SPARQL queries into your Python code by using the *RDFLib* and *SPARQLWrapper* libraries. 

The following code accesses the DBPedia knowledge graph using its SPARQL endpoint, and returns the result of the SPARQL query requesting all the labels asserted to Amsterdam (test it!)  

In [5]:
# This code only works if you are online

from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?cityName
    WHERE { 
        <http://dbpedia.org/resource/Amsterdam> rdfs:label ?cityName 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["cityName"]["value"])  

Amsterdam
أمستردام
حكومة أمستردام
Amsterdam
Amsterdam
Άμστερνταμ (δήμος)
Άμστερνταμ
Amsterdam
Amsterdamo
Ámsterdam
Amsterdam
Amsterdam (commune)
Amsterdam
Amstardam
Amsterdam
アムステルダム
Amsterdam
암스테르담
Amsterdam (gemeente)
Amsterdam
Amsterdam
Amesterdão
Амстердам
Amsterdam
Амстердам
阿姆斯特丹


Your task is now the following:
1. Write a SPARQL query that extracts all the cities from your local knowledge graph (constructed by loading the file example-from-slides.ttl) 
2. Find the number of inhabitants of these cities and the longitude and latitude information (if available) from DBPedia.
3. Merge the triples from example-from-slides.ttl with the information extracted from DBpedia + Save all these triples into a new file 'extended-example.ttl' + Print all triples in Turtle Syntax.

For your convenience, we already wrote the following functions that might be useful to complete this task. 
In addition, we have loaded and printed the 'example-from-slides.ttl' dataset.

In [159]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef, XSD
from SPARQLWrapper import SPARQLWrapper, JSON


# Loads the data from a certain file given as input in Turtle syntax into the Graph g  
# -------------------------
def load_graph(graph, filename):
    with open(filename, 'r') as f:
        graph.parse(f, format='turtle')
        

# Prints a certain graph given as input in Turtle syntax
# -------------------------
def serialize_graph(myGraph):
     print(myGraph.serialize(format='turtle'))
        

# Saves the Graph g in Turtle syntax to a certain file given as input
# -------------------------
def save_graph(myGraph, filename):
    with open(filename, 'w') as f:
        myGraph.serialize(filename, format='turtle')
        
    
# Changes the namespace of a certain URI given as input to a DBpedia URI 
# Example: transformToDBR("http://example.com/kad2020/Amsterdam") returns "http://dbpedia.org/resource/Amsterdam"
# -------------------------
def transformToDBR(uri):
    if isinstance(uri, Literal):
        # changes the literal to uppercase so that the object with the same name refers to an object and not the string
        return uri.upper()
    components = g.namespace_manager.compute_qname(uri)
    return "http://dbpedia.org/resource/%s"%(components[2])

# -------------------------

g = Graph()
load_graph(g, 'example-from-slides.ttl')
serialize_graph(g)


# Don't forget to run this cell before continuing the task.


@prefix ex: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City ;
    rdfs:subPropertyOf ex:contains .

ex:Capital rdfs:subClassOf ex:City .

ex:City rdfs:subClassOf ex:Location .

ex:Country rdfs:subClassOf ex:Location .




### 1. Write a SPARQL query that finds all the cities in the dataset

As you cannot directly use class City, you will have to find those cities in the dataset (example-from-slides.ttl) using implicit information that can be deduced from the domain and ranges of the relations (e.g. things in a hasCapital relation are capitals and a capital is a city, etc.).

Save all the cities returned from the SPARQL query into the empty set "cities". 

In [8]:
cities = set()

# Your code here
for s,p,o in g:
    p = p.split("/")
    if p[-1] == 'hasCapital':
        cities.add(o)
    elif p[-1] == 'containsCity':
        cities.add(o)
    
    
for city in cities:
    print(city) 
    

http://example.com/kad/Rotterdam
http://example.com/kad/Berlin
http://example.com/kad/Amsterdam


### 2. For each city, find from DBpedia its longitude & latitude, and its number of inhabitants (if available)

Don't forget to adapt the namespace of the cities in your dataset when querying DBpedia, using the above function *transformToDBR(uri)*. Also note that namespaces should never use the *https* protocol.

The empty graph h should only contain the triples extracted from DBpedia, but added to the URIs with the 'ex' namespace. 
An example of a triple in h is the following triple: 
       
       ex:Amsterdam dbo:populationTotal "872680"^^xsd:nonNegativeInteger .

In [164]:
h = Graph()

# Your code here
ex = Namespace("http://example.com/kad/")
geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
dbo = Namespace("http://dbpedia.org/ontology/")
h.bind('ex', ex)
h.bind('geo', geo)
h.bind('dbo', dbo)

for city in cities:
    old = city
    city = transformToDBR(city)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    qry = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
        SELECT DISTINCT ?cityName ?lat ?long ?population
        WHERE { """
    qry = qry + '\n\t\t<' + city + "> rdfs:label ?cityName ." + '\n\t\t<' + city + '> geo:lat ?lat .' + '\n\t\t<' + city + '> geo:long ?long .' + '\n\t\tOPTIONAL{\n\t\t<' + city + '> dbo:populationTotal ?population}' + '\n\t\tFILTER langMatches(lang(?cityName),"en")\n\t}\n GROUP BY ?cityName'
    # i used a filter to get rid of city duplicates in different languages. if these are needed, remove the filter section from the qry string above
    sparql.setQuery(qry)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        cityName = (result["cityName"]["value"])
        cityLat = (result["lat"]["value"])
        cityLong = (result["long"]["value"])
        cityPop = (result["population"]["value"])
        h.add((old, geo.lat, Literal(cityLat)))
        h.add((old, geo.long, Literal(cityLong)))
        if cityPop:
            h.add((old, dbo.populationTotal, Literal(cityPop)))
        
serialize_graph(h)


@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix ex: <http://example.com/kad/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .

ex:Amsterdam dbo:populationTotal "872680" ;
    geo:lat "52.3667" ;
    geo:long "4.9" .

ex:Berlin dbo:populationTotal "3769495" ;
    geo:lat "52.52" ;
    geo:long "13.405" .

ex:Rotterdam dbo:populationTotal "651157" ;
    geo:lat "51.9167" ;
    geo:long "4.5" .




### 3. Save your results

- Merge the triples from example-from-slides.ttl with the information extracted from DBpedia
- Save all these triples into a new file 'extended-example.ttl'
- Print all triples in Turtle Syntax.


In [75]:
# Your code here
h2 = Graph()
h2 = g + h

serialize_graph(h2)



@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix ex: <http://example.com/kad/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    dbo:populationTotal "872680" ;
    ex:closeBy ex:Germany ;
    geo:lat "52.3667" ;
    geo:long "4.9" .

ex:Belgium a ex:Country .

ex:Berlin dbo:populationTotal "3769495" ;
    geo:lat "52.52" ;
    geo:long "13.405" .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:Rotterdam dbo:populationTotal "651157" ;
    geo:lat "51.9167" ;
    geo:long "4.5" .

ex:closeBy rdfs:do

## Task 2: (3 points)  Implement Basic Inferencing Rules 

In the lecture we showed that the RDFS inference rules can be used to infer new knowledge. For example, infer class membership based on rdfs:domain or infer relationships between subjects and objects based on rdfs:subPropertyOf. 

Create rules to inference class membership based on the RDF Schema language features 
*	For example: infer that an instance belongs to a class because of domain and range restrictions
*	For example: infer that an instance belongs to a (super)class because it also belongs to a subclass

We implemented the rdfs2 rule. You should implement the 5 following remaining rules:  

*     (rdfs2) If G contains the triples (aaa rdfs:domain xxx.) and (uuu aaa yyy.)  then infer the triple (uuu rdf:type xxx.)
*     (rdfs3) If G contains the triples (aaa rdfs:range xxx.) and (uuu aaa vvv.) then infer the triple (vvv rdf:type xxx .)
*     (rdfs5) If G contains the triples (uuu rdfs:subPropertyOf vvv.) and (vvv rdfs:subPropertyOf xxx.) then infer the triple (uuu rdfs:subPropertyOf xxx.) 
*     (rdfs7) If G contains the triples (aaa rdfs:subPropertyOf bbb.) and (uuu aaa yyy.) then infer the triple (uuu bbb yyy) 
*     (rdfs9) If G contains the triples (uuu rdfs:subClassOf xxx.) and (vvv rdf:type uuu.) then infer the triple (vvv rdf:type xxx.) -> this one was not mentioned in the lecture, but is a very important one. 
*     (rdfs11) If G contains the triples (uuu rdfs:subClassOf vvv.) and (vvv rdfs:subClassOf xxx.) then infer the triple (uuu rdfs:subClassOf xxx.)


Run your rule reasoner on your knowledge graph.

In [139]:
def myRDFSreasoner(myGraph):
    inferredTriples = 0
    for sbj, prd, obj in myGraph:

        # --- rdfs2 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#domain"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 2) ", s, "rdf:type", obj)
        
        # --- rdfs3 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#range"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 3) ", o, "rdf:type", obj)

        # --- rdfs5 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.predicate_objects(URIRef(obj))
            for p, o in generator:
                if (p.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
                    inferredTriples += 1
                    print("(rdfs 5) ", sbj, "rdfs:subPropertyOf", o)
        
        # --- rdfs7 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 7) ", s, "ex:" + obj.split("/")[-1], o)
        
        # --- rdfs9 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.subject_predicates(URIRef(sbj))
            for s, p in generator:
                if (p.eq(URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"))):
                    inferredTriples += 1
                    print("(rdfs 9) ", s, "rdf:type", obj)
        
        # --- rdfs11 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.predicate_objects(URIRef(obj))
            for p, o in generator:
                if (p.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
                    inferredTriples += 1
                    print("(rdfs 11) ", sbj, "rdfs:subClassOf", o)
        
        
    print("---------------------------------")
    print("Number of inferred triples:", inferredTriples)
    print("---------------------------------")
    
myRDFSreasoner(g)



(rdfs 7)  http://example.com/kad/Netherlands ex:closeBy http://example.com/kad/Belgium
(rdfs 9)  http://example.com/kad/Germany rdf:type http://example.com/kad/Country
(rdfs 11)  http://example.com/kad/EuropeanCountry rdfs:subClassOf http://example.com/kad/Location
(rdfs 3)  http://example.com/kad/Germany rdf:type http://example.com/kad/Location
(rdfs 2)  http://example.com/kad/Netherlands rdf:type http://example.com/kad/Country
(rdfs 3)  http://example.com/kad/Rotterdam rdf:type http://example.com/kad/City
(rdfs 5)  http://example.com/kad/hasCapital rdfs:subPropertyOf http://example.com/kad/contains
(rdfs 7)  http://example.com/kad/Netherlands ex:containsCity http://example.com/kad/Amsterdam
(rdfs 7)  http://example.com/kad/Germany ex:containsCity http://example.com/kad/Berlin
(rdfs 9)  http://example.com/kad/Netherlands rdf:type http://example.com/kad/Location
(rdfs 9)  http://example.com/kad/Belgium rdf:type http://example.com/kad/Location
(rdfs 2)  http://example.com/kad/Amsterdam 

## Task 3: (2 points) Build your very own RDFS knowledge graph. 


Define a small RDF Schema vocabulary in Turtle. You can choose your own domain (e.g. movies, geography, sports) respecting all the following rules:
*	The schema should define at least 4 classes, 4 properties, and 4 instances.
*   The properties should be used to relate the instances
*	The instances should be a member of your classes
*	All resources should have an rdfs:label in a suitable language.

You should use (at least) the following language features of RDF and RDFS:
* 	rdf:type (or 'a')
* 	rdfs:subClassOf
* 	rdfs:subPropertyOf
* 	rdfs:domain and rdfs:range
*	rdfs:label

Be sure to define the 'rdf:' and 'rdfs:' namespace prefixes for RDF and RDF Schema in your file (perhaps have a look at http://prefix.cc)

For creating your vocabulary, you can either use a text editor, or add the axioms directly (programatically) to your Knowledge Graph as you did last week. 

Play around with the inference rules you have created in the previous task to make sure that you some added some implicit knowledge, that becomes "visible" via inferencing (this will be useful for the next task). 

Finally:
- Add the knowledge you created into the RDFLIB graph datastructure *myRDFSgraph*, 
- Print *myRDFSgraph* in Turtle so that we can check your "design"
- Save *myRDFSgraph* into a new file 'myRDFSgraph.ttl'

In [165]:
myRDFSgraph = Graph()

# Your code here.
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
ex = Namespace("http://example.com/kad/")
dbo = Namespace("http://dbpedia.org/ontology/")

myRDFSgraph.bind('ex', ex)
myRDFSgraph.bind('rdf', rdf)
myRDFSgraph.bind('rdfs', rdfs)

myRDFSgraph.add((ex.animal, rdf.type, ex.Class))
myRDFSgraph.add((ex.fish, rdfs.subClassOf, ex.animal))
myRDFSgraph.add((ex.mammal, rdfs.subClassOf, ex.animal))
myRDFSgraph.add((ex.marsupial, rdfs.subClassOf, ex.mammal))

myRDFSgraph.add((ex.livesIn, rdfs.range, ex.location))
myRDFSgraph.add((ex.livesIn, rdfs.domain, ex.animal))
myRDFSgraph.add((ex.hasDiet, rdfs.domain, ex.animal))

myRDFSgraph.add((ex.nationalAnimalOf, rdfs.subPropertyOf, ex.nativeOf))
myRDFSgraph.add((ex.nativeOf, rdfs.subPropertyOf, ex.livesIn))

myRDFSgraph.add((ex.giantPanda, rdfs.label, Literal("Giant panda", lang="en")))
myRDFSgraph.add((ex.giantPanda, rdf.type, ex.mammal))
myRDFSgraph.add((ex.giantPanda, ex.nationalAnimalOf, ex.china))

myRDFSgraph.add((ex.whaleShark, rdf.type, ex.fish))
myRDFSgraph.add((ex.whaleShark, rdfs.label, Literal("Whale shark", lang="en")))
myRDFSgraph.add((ex.whaleShark, ex.hasDiet, ex.plankton))

myRDFSgraph.add((ex.koala, rdfs.label, Literal("Koala", lang="en")))
myRDFSgraph.add((ex.koala, rdf.type, ex.Marsupial))
myRDFSgraph.add((ex.koala, ex.nativeOf, ex.australia))

myRDFSgraph.add((ex.tarantula, rdfs.label, Literal("Tarantula", lang="en")))
myRDFSgraph.add((ex.tarantula, ex.livesIn, ex.southAmerica))

print("Now let's check what we can infer from your knowledge graph...")
print("The more rules you cover, the better!")
myRDFSreasoner(myRDFSgraph)

myRDFSgraph.serialize(destination="myRDFSgraph.ttl")



Now let's check what we can infer from your knowledge graph...
The more rules you cover, the better!
(rdfs 3)  http://example.com/kad/southAmerica rdf:type http://example.com/kad/location
(rdfs 11)  http://example.com/kad/marsupial rdfs:subClassOf http://example.com/kad/animal
(rdfs 9)  http://example.com/kad/giantPanda rdf:type http://example.com/kad/animal
(rdfs 9)  http://example.com/kad/whaleShark rdf:type http://example.com/kad/animal
(rdfs 7)  http://example.com/kad/koala ex:livesIn http://example.com/kad/australia
(rdfs 2)  http://example.com/kad/whaleShark rdf:type http://example.com/kad/animal
(rdfs 2)  http://example.com/kad/tarantula rdf:type http://example.com/kad/animal
(rdfs 5)  http://example.com/kad/nationalAnimalOf rdfs:subPropertyOf http://example.com/kad/livesIn
(rdfs 7)  http://example.com/kad/giantPanda ex:nativeOf http://example.com/kad/china
---------------------------------
Number of inferred triples: 9
---------------------------------


<Graph identifier=N4e3729fb95a34e57b77a5dfc48cefb39 (<class 'rdflib.graph.Graph'>)>

## Task 4 (2 points) Compare local inferences with GraphDB results

Upload *myRDFSgraph.ttl* to GraphDB (check [the GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md) before starting to work with GraphDB).

Formulate two different SPARQL queries, and write a Python code that executes these queries over your GraphDB SPARQL endpoint (check example of Task 1).

**Each SPARQL query should return a different type of inferred knowledge** (at least one triple that was not explicitly asserted in the graph).

Specify below next to your query, which type of RDFS rule is the GraphDB reasoner using to infer this answer (rdfs2, rdfs3, rdfs5, rdfs7, rdfs9, rdfs11). 

In [166]:
# Get your GraphDB repository URL (setup -> repositories -> repository url) and assign it to the variable 'myEndpoint' below. 
# It should be similar to this: 

myEndpoint = "http://DESKTOP-DMHOCIP:7200/repositories/KnowledgeAndData"  # KnowledgeAndData is the name of the repository
sparql = SPARQLWrapper(myEndpoint)

In [167]:
# Query 1 - Specify which RDFS rule are you testing: 
# I am testing rdfs7 (for koala) and rdfs5 (and rdfs7) (for panda)
# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ex: <http://example.com/kad/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?animalName ?location
    WHERE {
        ?animalName ex:livesIn ?location
    }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["animalName"]["value"], "lives in", result["location"]["value"]) 



http://example.com/kad/giantPanda lives in http://example.com/kad/china
http://example.com/kad/koala lives in http://example.com/kad/australia
http://example.com/kad/tarantula lives in http://example.com/kad/southAmerica


In [168]:
# Query 2 - Specify which RDFS rule are you testing: 
# I am testing rdfs11 (koala) and rdfs9 (panda and whaleshark)
# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ex: <http://example.com/kad/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?animalName
    WHERE {
        ?animalName rdf:type ex:animal
    }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["animalName"]["value"]) 



http://example.com/kad/giantPanda
http://example.com/kad/koala
http://example.com/kad/tarantula
http://example.com/kad/whaleShark
